# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [141]:
import json
with open('/data/negotiated_rates.json') as f:
    data= json.load(f)
def average_rate(file):
    
    totalamount = 0
    totalcount = 0
    for item in data["out_of_network"]:
        if "allowed_amounts" not in item:
            continue
        for allowed_amount in item["allowed_amounts"]:
            if "payments" not in allowed_amount:
                continue
            if "allowed_amount" not in allowed_amount["payments"]:
                continue
            totalamount += allowed_amount["payments"]["allowed_amount"]
            totalcount += 1
    if totalcount > 0:
        return round(totalamount/totalcount, 2)
    else:
        return 0.00
    

In [142]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [148]:
import json
with open('/data/negotiated_rates.json') as f:
    data = json.load(f)
def get_rate(file, billing_code, service_code):    
    amount = None
    for item in data["out_of_network"]:
        if item["billing_code"] == billing_code:
            if "allowed_amounts" not in item:
                continue
            for allowed_amount in item["allowed_amounts"]:
                if allowed_amount["service_code"] == service_code:
                    if "payments" not in allowed_amount:
                        continue
                    if "allowed_amount" not in allowed_amount["payments"]:
                        continue
                    amount = allowed_amount["payments"]["allowed_amount"]
                    break
    return amount

    

In [149]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [150]:
from datetime import datetime
import json
with open('/data/negotiated_rates.json') as f:
    data = json.load(f)
    
def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    
    Amount = None
    for item in data["out_of_network"]:
        if item["billing_code"] == billing_code:
            if "allowed_amounts" not in item:
                continue
            for allowed_amount in item["allowed_amounts"]:
                if allowed_amount["service_code"] == service_code:
                    if "payments" not in allowed_amount:
                        continue
                    if "allowed_amount" not in allowed_amount["payments"]:
                        continue
                    Amount = allowed_amount["payments"]["allowed_amount"]
                    if pat_age >= 65:
                        Amount*= .5
                    elif datetime.strptime(visit_date, "%Y-%m-%d").weekday() == 0:
                        Amount *= .75
                    return round(Amount, 2)
        return None
    

In [151]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [155]:
import json
import csv
from datetime import datetime
with open('/data/negotiated_rates.json') as f:
    data = json.load(f)  
    
def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    
    Amount = None
    for item in data["out_of_network"]:
        if item["billing_code"] == billing_code:
            if "allowed_amounts" not in item:
                continue
            for allowed_amount in item["allowed_amounts"]:
                if allowed_amount["service_code"] == service_code:
                    if "payments" not in allowed_amount:
                        continue
                    if "allowed_amount" not in allowed_amount["payments"]:
                        continue
                
                    Amount = allowed_amount["payments"]["allowed_amount"]
                    if pat_age >= 65:
                        Amount *= .5
                    elif datetime.strptime(visit_date, "%Y-%m-%d").weekday() == 0:
                        Amount *= .75

                    return round(Amount, 2)

    return None

def summarize_reimbursement(csv_file, json_file):

    visits = open(csv_file)
    f = csv.reader(visits)
    fields = next(f)
    
    by_month = {}
    by_hospital = {}
    for row in f:
        result = get_adjusted_rate(json_file, row[4], row[5], int(row[3]), row[6])
        if result is None:
            continue
        
        month = row[6][:7]
        if month not in by_month:
            by_month[month] = result
        else:
            by_month[month] += result
        
        if row[1] not in by_hospital:
            by_hospital[row[1]] = result
        else:
            by_hospital[row[1]] += result
        
        # Round to 2 decimal places
        for k, v in by_month.items():
            by_month[k] = round(v, 2)
        for k, v in by_hospital.items():
            by_hospital[k] = round(v, 2)
    
    return [by_month, by_hospital]


In [127]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

[{'2021-05': 192.38,
  '2021-03': 378.72,
  '2021-07': 277.67,
  '2021-06': 236.53,
  '2021-11': 229.7,
  '2021-10': 234.52,
  '2021-12': 297.87,
  '2021-04': 337.7,
  '2021-09': 160.4,
  '2021-01': 111.91,
  '2021-02': 158.55,
  '2021-08': 152.28},
 {'Missouri Baptist': 514.18,
  'SSM DePaul': 460.02,
  'SLU Hospital': 409.67,
  'Barnes Jewish': 485.7,
  'Mercy Springfield': 518.59,
  'Mercy St. Louis': 380.07}]

In [128]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)